In [54]:
import pandas as pd

def generate_IV(diode_param, Temp, datapoints, Ncell, Voc=None, Isc=None):
    """
    This function will generate I-V data points given all the 5 diode parameters 
    
    Input  : 5 parameters: Rs, Rsh, Isat, Iph, eta
             Voc, Temperature(Celcius), N(Number of data points), Ncell
             
    Output : Returns a dictionary of I-V data (size = N) and metadata (data, metadata)
    
    This function will spit out I-V data points. 
    
    ** This auto generates Voc and Isc also if not provided **
    **Temperature is just taken to calculate Vt**
    
    """
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Vt = kb*(Temp+273)/q
    
    Rs, Rsh, Isat, Iph, eta = diode_param
    
    # If the Voc argument is not passed explicitly, this will calculate Voc also from the 5 parameters
    if not Voc:             # If Voc=None
        Voc_implicit_eqn = lambda voc: Iph - Isat*(np.exp((voc) / (eta*Ncell*Vt))-1) - (voc)/Rsh # i=0 @Voc
        Vocr = root(Voc_implicit_eqn,50)
        Voc = np.round(Vocr.x[0], 2)
        #print ('Auto generated Voc is :', Voc)
    
    # If the Isc argument is not passed explicitly, this will calculate Isc also from the 5 parameters
    if not Isc:             # If Voc=None
        Isc_implicit_eqn = lambda isc: -isc + Iph - Isat*(np.exp((isc*Rs) / (eta*Ncell*Vt))-1) - (isc*Rs)/Rsh # v=0 @Isc
        Isc = np.round(root(Isc_implicit_eqn,50).x[0], 2)
        #print ('Auto generated Isc is :', Isc)    
    
    # Now solving the implicit equation of the solar cell 
    I_implicit_equn = lambda i: -i + Iph - Isat*(np.exp((v+i*Rs) / (eta*Ncell*Vt))-1) - (v+i*Rs)/Rsh
    V = np.linspace(0,Voc,datapoints)  #Generating the Voltage vector
    I = []
    for v in V:                        # For each value of Voltage, current is calculated
        ior = root(I_implicit_equn,10)
        io = ior.x[0]
        I.append(io)
    
    I = np.round(I, 2)                 # Rounding upto 2 places
    V = np.round(V, 2)
    
    ## Now calculating Pmax, FF, Im, Vm
    df = pd.DataFrame(dict(V=V, I=I))
    df['P'] = df['I']*df['V']
    values = df.loc[df['P'].idxmax()]
    Vm, Im, Pmax = values
    FF = np.round(Pmax/(Voc*Isc),2)
    
    #metadata = dict(Voc=Voc, Isc=Isc, Pmax=np.round(Pmax,2), Vmp=Vm, Imp=Im, FF=FF) 
    metadata = dict(Voc=[Voc], Isc=[Isc], Pmax=[np.round(Pmax,2)], Vmp=[Vm], Imp=[Im], FF=[FF]) 
    data = dict(x=V, y=I)
    
    return (data, metadata)              # Returning a dictionary of data and metadata

In [55]:
from bokeh.models import Slider, CustomJS
from bokeh.models.widgets import Button, TextInput, DataTable, TableColumn
from bokeh.layouts import row, column, gridplot
from bokeh.plotting import Figure, output_notebook, show, ColumnDataSource, output_file
import pandas as pd
from scipy.optimize import root
import numpy as np
from os.path import dirname, join

output_notebook()

def plot_iv(doc):
    
    diode_param = (0.3551735739069161, 20051.491203008736, 2.9778179325966427e-10, 9.250163845946686, 1.0266558791833786)
    data_pair = generate_IV(diode_param, 25, 100, 72)
    source = ColumnDataSource(data=data_pair[0])
    
    metasource = ColumnDataSource(data=data_pair[1])
    col = [
        TableColumn(field="Voc", title="Voc"),
        TableColumn(field="Isc", title="Isc"),
        TableColumn(field="Pmax", title="Pmax"),
        TableColumn(field="Vmp", title="Vmp"),
        TableColumn(field="Imp", title="Imp"),
        TableColumn(field="FF", title="Fill Factor"),
    ]
    meta_table = DataTable(source=metasource, columns=col)
    
    
    plot = Figure(y_range = (-1, 10), x_range = (0,60))
    plot.xaxis.axis_label = 'Voltage (V)'
    plot.yaxis.axis_label = 'Current (I)'
    plot.scatter('x', 'y', source=source, line_width=3, line_alpha=0.6)
    
    Rs_input = TextInput(value='0.35', title='Series Resistance')
    Rsh_input = TextInput(value='20051', title='Shunt Resistance')
    Isat_input = TextInput(value='2.97e-10', title='Isat')
    Iph_input = TextInput(value='9.25', title='Iph')
    eta_input = TextInput(value='1.02', title='Eta (Ideality Factor)')
    datasize_input = TextInput(value='100', title='Data Size')
    submit = Button(label='Submit', button_type='success')
    dwnld = Button(label='Download (csv)',)
    
    def get_inputs():
        return ((float(Rs_input.value) ,float(Rsh_input.value) ,float(Isat_input.value) ,float(Iph_input.value) ,float(eta_input.value)),float(datasize_input.value))
    
    def update_plot(event):
        params, size = get_inputs()
        data_pair = generate_IV(params, 25, size, 72)
        source.data = data_pair[0]
        metasource.data = data_pair[1]

    def download():
        df = pd.DataFrame(source.data)
        df.to_csv('iv-data.csv')
        
    submit.on_click(update_plot)
    dwnld.on_click(download)
    
    layout = row(plot, column(Rs_input, Rsh_input, Isat_input, Iph_input, eta_input, datasize_input, submit, dwnld),)
    doc.add_root(column(layout, meta_table))

show(plot_iv)

Loading BokehJS ...